#INIT

In [0]:
from pyspark.sql.functions import col, lit, concat

COVERED_DATE = {
    "start": "2025-01-01",
    "end": "2025-01-31"
}

#Read From Silver Loans and Join them in Bronze Loancharges

In [0]:
query = f"""
        SELECT
            blc.Id AS id,
            blc.LoanId AS loanid,
            blc.BranchCode AS branchcode,
            bls.Name AS loanschemename,
            sl.status AS status,
            blc.LoanSchemeChargeId AS loanschemechargeid,
            blct.Code AS loanchargetypecode,
            blct.Name AS loanchargename,
            blc.ReferenceCode AS referencecode,
            blc.Remarks AS remarks,
            blct.AccountCode AS accountcode,
            blct.LoanTranxType AS loantranxtype,
            sl.loandate AS loandate,
            blc.Amount AS amount,
            blc.Deleted AS deleted,
            blc.LastModified AS lastmodified
        FROM workspace.silver.silver_loans sl
        LEFT JOIN workspace.bronze.bronze_loancharges blc
            ON sl.id = blc.LoanId
        LEFT JOIN workspace.bronze.bronze_loanschemecharges blsc
            ON blc.LoanSchemeChargeId = blsc.Id
        LEFT JOIN workspace.bronze.bronze_loanchargetypes blct
            ON blsc.LoanChargeTypeId = blct.Id
        LEFT JOIN workspace.bronze.bronze_loanschemes bls
            ON blsc.LoanSchemeId = bls.Id
        WHERE sl.loandate >= DATE('{COVERED_DATE["start"]}')
        AND sl.loandate < DATE('{COVERED_DATE["end"]}') + INTERVAL 1 DAY
"""

loancharges_df = spark.sql(query)

#Data Transformations

##Assign Charge Group Base on Loantanxtype and Account Code

In [0]:
loancharges_df = loancharges_df.withColumn("chargegrp", concat(lit("C0"), col("loantranxtype"), lit("_"), col("accountcode")))

#Write to Silver Loancharges


In [0]:
loancharges_df.write.mode("overwrite").option("overwriteSchema", "true").saveAsTable("silver.silver_loancharges")